## Defining a Service: Server and Client

Now that the message types and the service have been defined, lets write a simple server and client called *version_service*

In [ ]:
import os
import subprocess

home = os.path.expanduser('~')
workspace = home +'/ros2_ws/'
sourceFiles = workspace + 'src/'
serviceName = 'version_service'
servicePath = sourceFiles + serviceName

os.chdir(sourceFiles)
if not os.path.exists(servicePath):
    reply = subprocess.run(['ros2','pkg','create','--build-type',
        'ament_python',serviceName,'--dependencies','rclpy',
        'example_interfaces'],capture_output=True) #same as !ros2 pkg create --build-type ament_python py_srvcli --dependencies rclpy example_interfaces    


Next set up the server and client - assuming that you have already created the package with the service and message types

In [ ]:
os.chdir(servicePath + '/' + serviceName)

In [ ]:
%%writefile VersionServer.py
from version.srv import Version                                                           

import rclpy
from rclpy.node import Node

class VersionServer(Node):
    def __init__(self):
        super().__init__('version_service')
        self.srv = self.create_service(Version, 'versionServer', self.version_callback)      

    def version_callback(self, request, response):
        response.value = 3.27 + request.mode
        self.get_logger().info('Requesting version') 

        return response

def main(args=None):
    rclpy.init(args=args)
    version_service = VersionServer()
    rclpy.spin(version_service)
    rclpy.shutdown()

if __name__ == '__main__':
    main()

In [ ]:
%%writefile VersionClient.py
from version.srv import Version                                                           
import sys
import rclpy
from rclpy.node import Node


class VersionClient(Node):  #async
    def __init__(self):
        super().__init__('version_client_async')
        self.cli = self.create_client(Version, 'versionServer')       
        while not self.cli.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('service not available, waiting again...')
        self.req = Version.Request()                                   

    def send_request(self, mode = 1):
        self.req.mode = int(mode)

        self.future = self.cli.call_async(self.req)

def main(args=None):
    rclpy.init(args=args)

    client = VersionClient()
    client.send_request(2)

    while rclpy.ok():
        rclpy.spin_once(client)
        if client.future.done():
            try:
                response = client.future.result()
            except Exception as e:
                client.get_logger().info('Service call failed %r' % (e,))
            else:
                client.get_logger().info('Version: %d' % (response.value))  
            break

    client.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()

And add them to the setup file

In [ ]:
os.chdir(servicePath)

In [ ]:
%%writefile setup.py
from setuptools import setup

package_name = 'version_service'

setup(
    name=package_name,
    version='0.0.0',
    packages=[package_name],
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='parallels',
    maintainer_email='parallels@todo.todo',
    description='TODO: Package description',
    license='TODO: License declaration',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
            'service = version_service.VersionServer:main',
            'client = version_service.VersionClient:main',
        ],
    },
)

In [ ]:
os.chdir(workspace)
subprocess.run(['colcon','build','--packages-select',serviceName],capture_output=True)


Now you can confirm in the Terminal that they were running 
```bash
cd ~/ros2_ws
export ROS_DOMAIN_ID=0
. install/setup.bash
python3.8 VersionServer.py
```
This will start the server running.  And then open a new terminal and try 
```bash
cd ~/ros2_ws
export ROS_DOMAIN_ID=0
. install/setup.bash
python3.8 VersionClient.py
```